### Utterances por frase sin simbolos


In [1]:
import pylangacq as pla
import pandas as pd
import spacy
import glob
import re

In [2]:
eve = pla.read_chat('./Corpus/Pitt/Control/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['control']*len(tagged))
control_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })

In [3]:
eve = pla.read_chat('./Corpus/Pitt/Dementia/cookie/*.cha', encoding = 'utf-8')
tagged = eve.tagged_sents(participant='PAR')
sentences = eve.sents(participant='PAR')
group = list(['dementia']*len(tagged))
dementia_df = pd.DataFrame(
    {'utterances': sentences,
     'tagged': tagged,
     'group': group
    })

In [7]:
# Unimos los dos df , reorganizamos random y pasamos la colummna utterances de list -> str
from sklearn.utils import shuffle

df = pd.concat([control_df, dementia_df])
df = shuffle(df).reset_index(drop=True)

utterances_str = []
for i in range(len(df['utterances'])):
    list1 = df['utterances'][i]
    str1 = ' '.join(list1)
    utterances_str.append(str1)
    
df['utterances_str'] = utterances_str
df.head()

,utterances,tagged,group,utterances_str
0,"[that's, about, all, .]","[(that's, PRO:DEM, that, (1, 2, SUBJ)), (CLITI...",dementia,that's about all .
1,"[what's, it, always, say, that, ?]","[(what's, PRO:INT, what, (1, 5, SUBJ)), (CLITI...",dementia,what's it always say that ?
2,"[I, don't, see, anything, else, happening, .]","[(I, PRO:SUB, I, (1, 4, SUBJ)), (don't, MOD, d...",control,I don't see anything else happening .
3,"[and, the, mother, is, washing, the, dishes, b...","[(and, COORD, and, (1, 5, LINK)), (the, DET:AR...",control,and the mother is washing the dishes but she's...
4,"[a, boy, is, getting, a, cookie, out_of, a, co...","[(a, DET:ART, a, (1, 2, DET)), (boy, N, boy, (...",control,a boy is getting a cookie out_of a cookie jar ...


In [6]:
data = df[['group', 'utterances_str']].rename(columns={"group":"label", "utterances_str":"text"})
data.head()

,label,text
0,control,is that it ?
1,dementia,and he's getting cookies .
2,control,the water's coming out_of the faucet .
3,dementia,she's doing the dishes .
4,dementia,and the woman is drying dishes .


In [9]:
data['label'].value_counts()

dementia    3908
control     3145
Name: label, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

df = data.copy()

X = df['text']  # this time we want to look at the text
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()

X_train_counts = count_vect.fit_transform(X_train)
X_train_counts.shape

(4725, 1403)

In [13]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(4725, 1403)

In [14]:
from sklearn.svm import LinearSVC
clf = LinearSVC()
clf.fit(X_train_tfidf,y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [17]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

# Feed the training data through the pipeline
text_clf.fit(X_train, y_train) 

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [18]:
# Form a prediction set
predictions = text_clf.predict(X_test)
# Report the confusion matrix
from sklearn import metrics
print(metrics.confusion_matrix(y_test,predictions))

[[615 445]
 [346 922]]


In [19]:
# Print a classification report
print(metrics.classification_report(y_test,predictions))

              precision    recall  f1-score   support

     control       0.64      0.58      0.61      1060
    dementia       0.67      0.73      0.70      1268

   micro avg       0.66      0.66      0.66      2328
   macro avg       0.66      0.65      0.65      2328
weighted avg       0.66      0.66      0.66      2328



In [20]:
# Print the overall accuracy
print(metrics.accuracy_score(y_test,predictions))

0.6602233676975945


### Utterances por dialogo con simbolos

In [4]:
nlp = spacy.load('en_core_web_sm')

error: bad escape \p at position 257